In [ ]:
"Apartado 1 ==> Descripciones básicas de implementación <=="

In [ ]:
"""
==> Generación de la población inicial con especial indicación del número de reglas por individuo consideradas <==

Para inicializar la poblacion hemos optado por marcar al usuario unos margenes en cuanto al numero de reglas y al numero
de individuos. En cuanto al primero, hemos tenido especial cuidado con los recursos de los que disponemos puesto, que como
ya sabemos este tipo de algoritmos requiere de un coste computacional alto, y ello se ha visto reflejado a la hora de hacer 
las pruebas. Por tanto, marcamos un número máximo de reglas posibles, y para cada individuo en particular,
empleamos las funciones de rand, buscando asi la mayor aleatoriedad posible.

Por otra parte, y como ya se comento en las diferentes clases, el uso de reglas donde todos los alelos valen 1 o 0, no
serán de gran ayuda y es por ello, que al crear cada regla del individuo, se verifica que esta condicion no se da,
y en caso de que se de, se muta uno de los alelos para asi conseguir ya una regla valida.

Cuando estos individuos ya estan creados y verificados todos ellos son añadidos a una población que se albergará y sobre 
la que se trabajará durante todas las pruebas, logicamente.
"""

In [ ]:
"""
==> Mecanismo de cruce implementado <==

En cuanto al cruce implementado, recordamos la posibilidad de realizar cruces con otras reglas de otros progenitores,
asi como intercambiar las reglas con otros progenitores. Para el primer tipo (que es la forma predeterminada de trabajar
de nuestro algoritmo). Se comprueba la probabilidad introducida por el usuario y se genera un número aleatorio entre 0 y 1. En caso de ser inferior a la porbabilidad marcada por el usuario, se realizará ese cruce. Es entonces donde verificamos cuantas reglas hay
por individuo, y escogemos aleatoriamente cual de esas serán las elegidas para el cruce y aleatoriamente escogemos el punto de cruce.

Por otra parte, para inter, el proceso a seguir es muy similar, excepto el último paso, donde no escogemos ahora el punto
de cruce, pues no es necesario, y en su lugar, se intercambian directamente las reglas al completo.
"""

In [ ]:
"""
==> Mecanismo de mutación implementado <==

Para desarrollar mutacón hemos implementado, 3 posibles tipos de mutación donde la más "común" que hemos usado para
las pruebas es la de bit-flip. Las otras dos recordamos que buscan añadir o eliminar una de las reglas del individuo,
con sus posteriores verificaciones. Si nos centramos en la primera, primero debemos verificar el número de reglas de
las que dispone el individuo, y decidir sobre cual de ellas se analizará la mutación de este tipo. A partir de ello
escogemos alelo a alelo una probabilidad aleatoria que de ser inferior a la probabilida de mutación escogida por el usuario
se realizará ese cambio en ese bit. Por último chequeamos que esta regla afectada, no es una regla unidad o completa de ceros
ya que de ser habra que modificar algún bit aleatoriamente para que sea válida.

Para el tipo de mutación en el que añadimos una regla, verificamos que dado el individuo concreto, este no tiene ya
el máximo posible de reglas que permite el algoritmo. De lo contrario, se verifica que la probabilidad de mutación sea inferior
a la indicada y a partir de este punto, consiste básicamente en un proceso similar a la inicialización de la población pero
con una única regla sobre un único individuo.
Por otro lado, el tipo de mutación donde se le elimina una regla parte de verificar que al menos el individuo en cuestión
alberga 2 reglas, para que al menos pueda seguir trabajando en la población con el mínimo requerido. Entonces, aleatoriamente
se escoge cual de estas será eliminada y se borra del progenitor."""

In [ ]:
"""
==> Mecanismo de clasificación implementado <==

Es aqui donde requerimos unos recursos suficientes, puesto que nuestra idea es clasificar todas las instancias correspondientes
con las reglas de un individuo. Conociendo esto de antemano, la función de clasificación se centra en
recoger todos los atributos del dataset y escogiendo cada instancia del fichero de datos, analizar para una regla completa,
cada parte de la misma con la parte correspondiente del dataset (recordemos en codificación One Hot). En caso de que todos los atributos de la regla se activen con los atributos del ejemplo del dataset,  se activaría la regla y se añade la clase de la regla a la lista de clases predichas. Finalmente, cuando ya se hana probado todas las reglas del individuo, se elige la clase a predecir mediante un voto por mayoría de todas las predicciones de las reglas.
Si se diera el caso de que ninguna regla del individuo se ha activado, la clase que se predeciría por defecto es 0.
"""

In [ ]:
"""Apartado 2 ==> Resultados de clasificación para los conjuntos de prueba"""

In [ ]:
"""Apartado 3 ==> Analisis de resultados"""

In [ ]:
"""
Importancia del numero de reglas

Como veremos más adelante el número de reglas, dadas las condiciones especificadas en la práctica es un pilar clave, para
obtener una solución óptima y sin necesidad de emplear un mayor número de individuos, o un mayor número de generaciones.
Es una forma muy útil de evitar desempates durante la clasificación y además supone tener a priori un mayor número de reglas,
que aún siendo escogidas aleatoriamente, dado el numero de atributos que las componenen, siempre será preferible, emplear
más reglas y no tantos individuos. Dado que estamos limitados a emplear entre 100 y 150 generaciones y estamos probando en muchas
ocasiones con unos 50 individuos, solemos especificar que el máximo de reglas este entre los 50 y 70, y con ello obtener
individuos muy variados y con una tasa de error bastante baja.

"""

In [ ]:
"""
Tamaño de la población

Como ya sabemos, la inicialización de la población puede resultar ser un criterior fundamental a la hora de emplear algoritmos
geéticos, pues dependerá de la naturaleza del problema, el que sean necesarios más o menos individuos. Normalmente,
una población inicial va desde los cientos a miles de inidividuos, y logicamente, se generá de forma aleatoria, de tal
modo que podamos cubrir todo el espectro de posibles soluciones para este problema concreto. Otra forma de decidir el número
de individuos es aprovechar el conocimiento del problema a solucionar, y entonces tratar de generar posibles inidivudos
de una forma concreta, para generar unas determinadas soluciones, que serán posiblemente más válidas frente a otras.

Si extrapolamos esto a nuestro contexto, nos daremos cuenta que, partimos de una cierta diferencia frente a lo dicho 
anteriormente, y es que, nosotros tenemos la posibilidad de aplicar más de una regla a un inidividuo. Con ello, puede 
no ser necesario generar una población de miles de individuos, sino generar los suficientes para que entre la suma de sus reglas
se abarque todas o casi todas las soluciones posibles.

Es por ello que nosotros hemos considerado oportuno QUE .... 

"""

In [ ]:
"""
Generaciones

Como es lógico pensar, cuantas más generaciones se empleen, más individuos se generarán, y aunque no todos sean válidos,
aumentará el pocentaje de individuos con mayor fitnness respecto los demás. Es decir, tendremos en cuenta un mayor número
de individuos con un fitnness superior. Podemos verlo como que aquellos individuos que tiene una mejor adaptación (un 
mayor fitnness) propagarán sus genes en sucesivas generaciones y con ello probablemente se conseguirá lo que conocemos
como "superindividuos". Por tanto, a lo largo de las generaciones las buenas características de cada individuo se van 
propagando a través de la población y empleando el cruce entre individuos, logramos explorar distintas areas de búsquedad,
que durante generaciones primitivas era imposible.

Para comprender aún más la importancia de las generaciones, podemos hacer uso de la definición de convergencia empleada
por Kenneth A. de Jong, donde nos comenta que la influencia del número de generaciones será un factor fundamental,
en la adaptación media extendida a todos los individuos de la población, asi como la adaptación del mejor individuo, que
iran convergiendo en un óptimo global.

Es decir, este concepto de convergencia de la progresión hace referencia a  cuando al menos el 95% de la población, comarten
el mismo valor para dicho gen. Por tanto, cuando todos los genes hayan convergido, la población general será óptima.

Por tanto damos por válido el uso de TANTAS GENERACIONES .... PORQUE ....
"""

In [ ]:
"""
Tasa de Cruce y Tasa de Mutación

Sabemos que el cruce es una parte fundamental al generar progenitores, y es por ello, que esa gran influencia que genera
sobre los descendientes hace que sea un factor muy delicado con el que trabajar. Con forme esta tasa sea más alta,
se dará una mayor diversidad en la descendencia y con ello, podemos estar alejandonos de esa convergencia que buscamos 
ansiadamente. Dicho de otro modo, es muy aconsejable no cruzar un determinado número de individuos, para evitar quedarnos
estancados en óptimos locales, y asi tratar de buscar en todo momento el óptimo global.

Por otro lado contamos con la tasa de mutación como una forma alternativa de salir de esos mínimos locales. Aún asi,
su capacidad de ajuste rara vez es bueno, puesto que no tienen en cuenta la historia del resto de generaciones. Por tanto,
consideramos que es preferebile no emplear una tasa alta de mutación para no introducir individuos no deseados.

Con ello, creemos que las tasas más aptas SON LAS DE ....
"""